In [1]:
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse
import re

# Problem 1

In [2]:
def scraping_books(url,base_url="http://books.toscrape.com/"):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    title = response.css("h3 > a::attr(title)").extract()
    price1 = response.css("p.price_color::text").extract()
    price = [float(i.replace("Â£", "")) for i in price1]
    burl = response.css("h3 >a::attr(href)").extract()
    purl = response.css("img::attr(src)").extract()
    star = response.css("p[class^='star-rating']::attr(class)").extract()
    star_rating = []
    for i in star:
        star_rating.append(i.replace("star-rating", ""))
    stock  =response.css("p.price_color ~ p[class^='instock']::attr(class)").extract()
    instock = [i.replace("availability", " ") for i in stock]    
    base_url = "http://books.toscrape.com/catalogue/"
    bookurl = [base_url + i for i in burl]
    picurl = [base_url + i for i in purl]
    genere = []
    bookinfo = []
    for i in bookurl:
        page = requests.get(i)
        response = TextResponse(body=page.text,url=i,encoding="utf-8")
        genere.append(response.css("li~li~li > a::text").extract())
        bookinfo.append((response.css("article[class='product_page'] > p::text")).extract())
    return pd.DataFrame({"title":title,  "price":price, "bookurl":bookurl, "picurl":picurl,"star_rating":star_rating,"instock":instock,'genere':genere,"bookinfo":bookinfo})

In [3]:
books = []
for i in range(1,51):
    cureent_page =scraping_books(url = f"http://books.toscrape.com/catalogue/page-{i}.html")
    if cureent_page.shape[0] == 0:
        break
    else:
        books.append(cureent_page)

In [4]:
books = pd.concat(books)
books

,title,price,bookurl,picurl,star_rating,instock,genere,bookinfo
0,A Light in the Attic,51.77,http://books.toscrape.com/catalogue/a-light-in...,http://books.toscrape.com/catalogue/../media/c...,Three,instock,[Poetry],[It's hard to imagine a world without A Light ...
1,Tipping the Velvet,53.74,http://books.toscrape.com/catalogue/tipping-th...,http://books.toscrape.com/catalogue/../media/c...,One,instock,[Historical Fiction],"[""Erotic and absorbing...Written with starling..."
2,Soumission,50.10,http://books.toscrape.com/catalogue/soumission...,http://books.toscrape.com/catalogue/../media/c...,One,instock,[Fiction],"[Dans une France assez proche de la nÃ´tre, un..."
3,Sharp Objects,47.82,http://books.toscrape.com/catalogue/sharp-obje...,http://books.toscrape.com/catalogue/../media/c...,Four,instock,[Mystery],"[WICKED above her hipbone, GIRL across her hea..."
4,Sapiens: A Brief History of Humankind,54.23,http://books.toscrape.com/catalogue/sapiens-a-...,http://books.toscrape.com/catalogue/../media/c...,Five,instock,[History],[From a renowned historian comes a groundbreak...
...,...,...,...,...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,55.53,http://books.toscrape.com/catalogue/alice-in-w...,http://books.toscrape.com/catalogue/../media/c...,One,instock,[Classics],[]
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,http://books.toscrape.com/catalogue/ajin-demi-...,http://books.toscrape.com/catalogue/../media/c...,Four,instock,[Sequential Art],[High school student Kei Nagai is struck dead ...
17,A Spy's Devotion (The Regency Spies of London #1),16.97,http://books.toscrape.com/catalogue/a-spys-dev...,http://books.toscrape.com/catalogue/../media/c...,Five,instock,[Historical Fiction],"[In Englandâs Regency era, manners and elega..."
18,1st to Die (Women's Murder Club #1),53.98,http://books.toscrape.com/catalogue/1st-to-die...,http://books.toscrape.com/catalogue/../media/c...,One,instock,[Mystery],"[James Patterson, bestselling author of the Al..."


In [5]:
books['price'].max()

59.99

In [6]:
books[books['price'] == books['price'].max()]['genere']

8    [Romance]
Name: genere, dtype: object

In [7]:
books['star_rating'].value_counts()

 One      226
 Three    203
 Two      196
 Five     196
 Four     179
Name: star_rating, dtype: int64

In [8]:
rating_map = {
    'Two': 2,
    'One': 1,
    'Five': 5,
    'Four': 4,
    'Three': 3
}

In [9]:
books['star_rating'] = books['star_rating'].apply(lambda x: x.strip())

In [10]:
books['star_rating'] =books['star_rating'].replace(rating_map)

In [11]:
books.corr()

,price,star_rating
price,1.000000,0.028166
star_rating,0.028166,1.000000


In [12]:
books.groupby('star_rating').mean()['price']

star_rating
1    34.561195
2    34.810918
3    34.692020
4    36.093296
5    35.374490
Name: price, dtype: float64

# Problem 2

def scraping_jobs(url):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    company_name = response.xpath("//p[@class='job_list_company_title']/text()").extract()
    job_name = response.xpath("//p[@class='font_bold']/text()").extract()
    location = response.xpath("////p[@class = 'job_location']/text()").re('[A-Z]+[a-z]+')
    deadlinen = response.xpath("//div[@class= 'job-inner job-list-deadline']/p[not(@class = 'job_location')]/text()").re("\d{2}\s[A-Z]+[a-z]+\s\d{4}")
    deadline = [i.replace('\n', '') for i in deadlinen]
    base_url = "https://staff.am"
    urls = response.xpath("//div[@class='web_item_card hs_job_list_item']/a/@href").extract()
    job_url = [base_url + i for i in urls]
    return pd.DataFrame({"company_name":company_name,"job_name":job_name,"location":location,"deadline":deadline,'job_url':job_url})


In [14]:
def scraping_jobs(url):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    company_name = response.xpath("//p[@class='job_list_company_title']/text()").extract()
    job_name = response.xpath("//p[@class='font_bold']/text()").extract()
    base_url = "https://staff.am"
    urls = response.xpath("//div[@class='web_item_card hs_job_list_item']/a/@href").extract()
    job_url = [base_url + i for i in urls]
    deadlinea = response.css("div[class = 'job-inner job-list-deadline'] >p:not([class='job_location'])")
    deadlinee =[i.css('::text').extract()[1] for i in deadlinea]
    deadline = [i.replace('\n'," ") for i in deadlinee]
    locatione = response.css("div[class = 'job-inner job-list-deadline'] >p[class='job_location']")
    locations = [i.css("::text").extract()[1] for i in locatione]
    location = [i.replace('\n',"").strip() for i in locations]
    return pd.DataFrame({"company_name":company_name,"job_name":job_name,'job_url':job_url,"deadline":deadline,'location':location})


In [15]:
jobs = []
for i in range(1,100):
    cureent_page =scraping_jobs(url = f"https://staff.am/en/jobs?page={i}&per-page=50")
    if cureent_page.shape[0] == 0:
        break
    else:
        jobs.append(cureent_page)

In [16]:
jobs= pd.concat(jobs)
jobs

,company_name,job_name,job_url,deadline,location
0,Chemonics International Inc.,"Public Communications and Outreach Officer, Ar...",https://staff.am/en/public-communications-and-...,08 July 2020,Yerevan
1,MOA Communications,Senior React.js Developer,https://staff.am/en/senior-reactjs-developer-62,30 July 2020,Yerevan
2,Chemonics International Inc.,"Senior Governance Officer, Armenia Integrity P...",https://staff.am/en/senior-governance-officer,08 July 2020,Yerevan
3,Chemonics International Inc.,"Deputy Chief of Party, Armenia Integrity Project",https://staff.am/en/deputy-chief-of-party,08 July 2020,Yerevan
4,Med Standard,Medical equipment representative,https://staff.am/en/medical-equipment-represen...,29 July 2020,Yerevan
...,...,...,...,...,...
37,ServiceTitan,"Senior Software Engineer, Front-End",https://staff.am/en/sr-software-engineer-front...,03 July 2020,Yerevan
38,ServiceTitan,Senior Site Reliability Engineer,https://staff.am/en/senior-site-reliability-en...,15 July 2020,Yerevan
39,ServiceTitan,Lead Software Engineer,https://staff.am/en/lead-software-engineer,05 July 2020,Yerevan
40,ServiceTitan,QA Automation Engineer,https://staff.am/en/qa-automation-engineer-91,05 July 2020,Yerevan


In [17]:
jobs['company_name'].value_counts()

Digitain                32
SoftConstruct           29
PicsArt                 23
ServiceTitan            16
TeamViewer Armenia      12
                        ..
Evocabank                1
«Teghout» CJSC           1
alvard vardanyan ltd     1
FCl Urartu SNGO          1
SmartCredit UCO LLC      1
Name: company_name, Length: 245, dtype: int64

In [18]:
jobs[jobs['job_name'].str.contains("Data")]

,company_name,job_name,job_url,deadline,location
25,buymie,Data analyst,https://staff.am/en/data-analyst-53,24 July 2020,Yerevan
3,Webb Fontaine Holding LLC,Data Engineer,https://staff.am/en/data-engineer-51,22 July 2020,Yerevan
41,ACBA-CREDIT AGRICOLE BANK CJSC,Junior Database Administrator,https://staff.am/en/junior-database-administra...,19 July 2020,Yerevan
35,Ucom CJSC,Senior Database Administrator,https://staff.am/en/senior-database-administra...,11 July 2020,Yerevan
25,Krisp,Data Specialist / QA Engineer,https://staff.am/en/data-specialist-qa-engineer-1,03 July 2020,Yerevan
44,BIVAC Armenia CJSC,Database Administrator,https://staff.am/en/database-administrator-83,02 July 2020,Yerevan
12,PicsArt,Data Engineer (Spark),https://staff.am/en/data-engineer-spark-1,30 June 2020,Yerevan
13,PicsArt,Data Scientist,https://staff.am/en/data-scientist-76,30 June 2020,Yerevan
35,DISQO,"Engineering Manager, Java Microservices & Data",https://staff.am/en/engineering-manager-java-m...,03 July 2020,Yerevan


In [19]:
url = "https://staff.am/en/jobs?page=1&per-page=50"
page = requests.get(url)
response = TextResponse(body=page.text,url=url,encoding="utf-8")
deadline = response.css("div[class = 'job-inner job-list-deadline'] >p[class='job_location']")
[i.css("::text").extract()[1] for i in deadline]

['\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nAlaverdi ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nNoyemberyan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ',
 '\nYerevan ']

In [20]:
deadline

[<Selector xpath="descendant-or-self::div[@class = 'job-inner job-list-deadline']/p[@class = 'job_location']" data='<p class="job_location">\n<i class="fa...'>,
 <Selector xpath="descendant-or-self::div[@class = 'job-inner job-list-deadline']/p[@class = 'job_location']" data='<p class="job_location">\n<i class="fa...'>,
 <Selector xpath="descendant-or-self::div[@class = 'job-inner job-list-deadline']/p[@class = 'job_location']" data='<p class="job_location">\n<i class="fa...'>,
 <Selector xpath="descendant-or-self::div[@class = 'job-inner job-list-deadline']/p[@class = 'job_location']" data='<p class="job_location">\n<i class="fa...'>,
 <Selector xpath="descendant-or-self::div[@class = 'job-inner job-list-deadline']/p[@class = 'job_location']" data='<p class="job_location">\n<i class="fa...'>,
 <Selector xpath="descendant-or-self::div[@class = 'job-inner job-list-deadline']/p[@class = 'job_location']" data='<p class="job_location">\n<i class="fa...'>,
 <Selector xpath="descendant-or-se

In [21]:
url = "https://staff.am/en/jobs?page=1&per-page=50"
page = requests.get(url)
response = TextResponse(body=page.text,url=url,encoding="utf-8")
a = response.xpath("//div[@class= 'job-inner job-list-deadline']/p[not(@class = 'job_location')]/text()").re("\d{2}\s[A-Z]+[a-z]+\s\d{4}")

In [22]:
v = []
for i in a:
    v.append(i.replace('\n', ""))
    

In [23]:
v

['30July2020',
 '08July2020',
 '08July2020',
 '29July2020',
 '20July2020',
 '08July2020',
 '30July2020',
 '15July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '08July2020',
 '08July2020',
 '12July2020',
 '07July2020',
 '15July2020',
 '30July2020',
 '30July2020',
 '30July2020',
 '12July2020',
 '29July2020',
 '15July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '12July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '29July2020',
 '08July2020',
 '15July2020',
 '15July2020']